In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import make_blobs
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import numpy as np

In [ ]:
# 1. Geração de Dados Sintéticos de Clientes
X, y_true = make_blobs(
    n_samples=400, 
    centers=4, 
    cluster_std=1.5, 
    random_state=42
)

In [ ]:
# Ajuste das escalas (Renda e Frequência)
X[:, 0] = np.abs(X[:, 0] * 10 + 50)  # Renda Anual
X[:, 1] = np.abs(X[:, 1] * 5 + 10)   # Frequência de Compra

In [ ]:
# 2. Treinamento do Modelo de Agrupamento Hierárquico Aglomerativo
n_clusters = 4
agg = AgglomerativeClustering(n_clusters=n_clusters)
labels = agg.fit_predict(X)

In [ ]:
# 3. Cálculo das "médias" de cada cluster (substitui centróides)
centroids = np.array([X[labels == i].mean(axis=0) for i in range(n_clusters)])

In [ ]:
# 4. Visualização dos Perfis Encontrados
plt.figure(figsize=(10, 7))
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap="tab10", s=50, alpha=0.6)
plt.scatter(centroids[:, 0], centroids[:, 1], 
            c="black", marker="X", s=300, 
            label="Perfis (Médias)", edgecolors='white')
plt.title("Segmentação Sintética de Clientes por Agrupamento Hierárquico")
plt.xlabel("Renda Anual (em milhares de R$)")
plt.ylabel("Frequência de Compra (Compras por Ano)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
# 5. Exibição do Dendrograma com corte automático
linked = linkage(X, method='ward')

# Encontrar a altura exata do corte para gerar n_clusters
clusters = fcluster(linked, n_clusters, criterion="maxclust")
# Pega a distância onde o corte separa os clusters
distancias = sorted(linked[:, 2], reverse=True)
cut_height = distancias[n_clusters-1]  # altura do corte automático

plt.figure(figsize=(12, 6))
dendrogram(
    linked,
    truncate_mode="lastp",  
    p=20,                   
    leaf_rotation=45,
    leaf_font_size=10,
    show_contracted=True,
    color_threshold=cut_height  # pinta cada grupo com cor diferente
)

plt.axhline(y=cut_height, c='red', linestyle='--', label=f"Corte para {n_clusters} Clusters")
plt.title("Dendrograma do Agrupamento Hierárquico (Ward)")
plt.xlabel("Clusters")
plt.ylabel("Distância (Inércia)")
plt.legend()
plt.show()

In [ ]:
# 6. Análise dos Perfis
print("\n--- Médias dos Perfis de Cliente Encontrados ---")
for i, (renda, frequencia) in enumerate(centroids):
    print(f"Perfil {i+1}: Renda Média ≈ R$ {renda:.2f} mil | Frequência Média ≈ {frequencia:.2f} Compras/Ano")